# Rob's sandbox

---

# Imports

#### Python modules

In [12]:
import sys

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import (
    train_test_split,
    cross_val_score
)
from sklearn.inspection import permutation_importance

# import matplotlib.pyplot as plt
# %matplotlib inline

import pandas as pd

# Ancillary modules

#### Support adjustments

In [2]:
sys.path.append("../..")

#### Modules

In [21]:
%load_ext autoreload
%autoreload 2

from src.utils.params import (
    param_grid,
    max_features,
    n_estimators,
    cv_rounds,
    evaluation_metric,
    transformation_pickle_loc,
    fe_pickle_loc
)

from src.pipelines.ingestion import (
    ingest
)

from src.pipelines.transformation import (
    transform
)

from src.pipelines.feature_engineering import (
    load_transformation,
    feature_generation,
    feature_selection,
    save_fe,
    feature_engineering
)

from proyecto_1 import (
    main
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


---

# Tests

## Feature engineering

#### Loading transformation pickle

In [4]:
df = load_transformation("../../" + transformation_pickle_loc)

In [5]:
df

,fecha_creacion,hora_creacion,dia_semana,codigo_cierre,incidente_c4,tipo_entrada,label,dia_inicio,mes_inicio,anio_inicio,hora_inicio,min_inicio
0,23/01/2016,22:35:04,Sábado,(A) La unidad de atención a emergencias fue de...,accidente-choque sin lesionados,BOTÓN DE AUXILIO,0.0,23,01,2016,22.0,35
1,23/01/2016,22:50:49,Sábado,(A) La unidad de atención a emergencias fue de...,accidente-choque con lesionados,BOTÓN DE AUXILIO,0.0,23,01,2016,22.0,50
2,24/01/2016,09:40:11,Domingo,(N) La unidad de atención a emergencias fue de...,accidente-choque sin lesionados,LLAMADA_911_066,1.0,24,01,2016,9.0,40
3,24/01/2016,22:40:57,Domingo,(N) La unidad de atención a emergencias fue de...,accidente-choque sin lesionados,LLAMADA_911_066,1.0,24,01,2016,22.0,40
4,24/01/2016,04:25:15,Domingo,(A) La unidad de atención a emergencias fue de...,accidente-choque sin lesionados,LLAMADA_911_066,0.0,24,01,2016,4.0,25
...,...,...,...,...,...,...,...,...,...,...,...,...
1383133,22/10/2020,16:17:05,Jueves,(A) La unidad de atención a emergencias fue de...,lesionado-atropellado,BOTÓN DE AUXILIO,0.0,22,10,2020,16.0,17
1383134,22/10/2020,16:29:13,Jueves,(A) La unidad de atención a emergencias fue de...,accidente-choque con lesionados,BOTÓN DE AUXILIO,0.0,22,10,2020,16.0,29
1383135,22/10/2020,12:00:26,Jueves,(D) El incidente reportado se registró en dos ...,accidente-choque sin lesionados,LLAMADA_911_066,0.0,22,10,2020,12.0,00
1383136,22/10/2020,12:01:25,Jueves,(D) El incidente reportado se registró en dos ...,accidente-choque con lesionados,LLAMADA_911_066,0.0,22,10,2020,12.0,01


#### Processing data for model

In [48]:
df_features_prc, df_labels = feature_generation(df)


++ Complete list of features (9) that will be fed to the model:
    1. hora_creacion
    2. dia_semana
    3. incidente_c4
    4. tipo_entrada
    5. dia_inicio
    6. mes_inicio
    7. anio_inicio
    8. hora_inicio
    9. min_inicio

++ List of categorical features (2) that will be processed through the pipeline are:
    1. incidente_c4
    2. tipo_entrada
{'incidente_c4': ['Detención ciudadana-accidente automovilístico', 'accidente-choque con lesionados', 'accidente-choque con prensados', 'accidente-choque sin lesionados', 'accidente-ciclista', 'accidente-ferroviario', 'accidente-monopatín', 'accidente-motociclista', 'accidente-otros', 'accidente-persona atrapada / desbarrancada', 'accidente-vehiculo atrapado', 'accidente-vehiculo desbarrancado', 'accidente-vehículo atrapado-varado', 'accidente-volcadura', 'cadáver-accidente automovilístico', 'cadáver-atropellado', 'detención ciudadana-accidente automovilístico', 'detención ciudadana-atropellado', 'lesionado-accidente automovilísti

In [27]:
df

,fecha_creacion,hora_creacion,dia_semana,codigo_cierre,incidente_c4,tipo_entrada,label,dia_inicio,mes_inicio,anio_inicio,hora_inicio,min_inicio
0,23/01/2016,22:35:04,Sábado,(A) La unidad de atención a emergencias fue de...,accidente-choque sin lesionados,BOTÓN DE AUXILIO,0.0,23,01,2016,22.0,35
1,23/01/2016,22:50:49,Sábado,(A) La unidad de atención a emergencias fue de...,accidente-choque con lesionados,BOTÓN DE AUXILIO,0.0,23,01,2016,22.0,50
2,24/01/2016,09:40:11,Domingo,(N) La unidad de atención a emergencias fue de...,accidente-choque sin lesionados,LLAMADA_911_066,1.0,24,01,2016,9.0,40
3,24/01/2016,22:40:57,Domingo,(N) La unidad de atención a emergencias fue de...,accidente-choque sin lesionados,LLAMADA_911_066,1.0,24,01,2016,22.0,40
4,24/01/2016,04:25:15,Domingo,(A) La unidad de atención a emergencias fue de...,accidente-choque sin lesionados,LLAMADA_911_066,0.0,24,01,2016,4.0,25
...,...,...,...,...,...,...,...,...,...,...,...,...
1383133,22/10/2020,16:17:05,Jueves,(A) La unidad de atención a emergencias fue de...,lesionado-atropellado,BOTÓN DE AUXILIO,0.0,22,10,2020,16.0,17
1383134,22/10/2020,16:29:13,Jueves,(A) La unidad de atención a emergencias fue de...,accidente-choque con lesionados,BOTÓN DE AUXILIO,0.0,22,10,2020,16.0,29
1383135,22/10/2020,12:00:26,Jueves,(D) El incidente reportado se registró en dos ...,accidente-choque sin lesionados,LLAMADA_911_066,0.0,22,10,2020,12.0,00
1383136,22/10/2020,12:01:25,Jueves,(D) El incidente reportado se registró en dos ...,accidente-choque con lesionados,LLAMADA_911_066,0.0,22,10,2020,12.0,01


In [70]:
l1 = list("abcde")
l1

['a', 'b', 'c', 'd', 'e']

In [71]:
c = ["c1", "c2", "c3"]
c

['c1', 'c2', 'c3']

In [72]:
for i in range(len(c)):
    l1.insert(i + l1.index("c"), c[i])
l1.remove("c")

In [73]:
l1

['a', 'b', 'c1', 'c2', 'c3', 'd', 'e']

#### Training, executing and evaluating model

In [7]:
model = RandomForestClassifier(oob_score=True, n_jobs=-1)

In [9]:
## Splitting data in train and test
X_train, X_test, y_train, y_test = train_test_split(df_features_prc, df_labels, test_size=0.3)

In [10]:
grid_search = GridSearchCV(model,
                           param_grid,
                           cv=2,
                           scoring=evaluation_metric,
                           return_train_score=True,
                           n_jobs=-1
                           )

grid_search.fit(X_train, y_train)

print(grid_search.best_params_)
print(grid_search.best_estimator_)

print("\n++ Grid search results:\n")
print("    ++++ Best estimator: {}".format(grid_search.best_estimator_))
print("    ++++ Number of features in best estimator: {} \n".format(grid_search.best_estimator_.n_features_))
print("    ++++ Best estimator oob score: {}\n".format(grid_search.best_estimator_.oob_score_))

{'criterion': 'gini', 'min_samples_leaf': 9, 'n_estimators': 500}
RandomForestClassifier(min_samples_leaf=9, n_estimators=500, n_jobs=-1,
                       oob_score=True)

++ Grid search results:

    ++++ Best estimator: RandomForestClassifier(min_samples_leaf=9, n_estimators=500, n_jobs=-1,
                       oob_score=True)
    ++++ Number of features in best estimator: 34 

    ++++ Best estimator oob score: 0.7956973587992514



#### Identifying most important variables

In [25]:
print(len(grid_search.best_estimator_.feature_importances_))
grid_search.best_estimator_.feature_importances_

34


array([0.00000000e+00, 8.29972427e-02, 2.71450590e-03, 3.61597570e-01,
       4.59154789e-04, 2.74016206e-05, 1.91502764e-05, 1.45341686e-02,
       1.34217488e-03, 1.84155936e-03, 1.59117930e-03, 1.32748621e-04,
       1.26035468e-03, 2.67931965e-02, 1.89707854e-03, 3.10741491e-03,
       1.02661983e-04, 1.20516769e-04, 1.04711518e-03, 6.83003257e-02,
       5.39169803e-04, 0.00000000e+00, 8.67861133e-05, 0.00000000e+00,
       9.35247886e-05, 2.47787875e-08, 3.59081942e-04, 9.93993505e-02,
       6.43209726e-03, 5.84196707e-03, 1.88749552e-01, 9.84750268e-02,
       2.61188926e-02, 4.01900572e-03])

In [17]:
feature_importance = pd.DataFrame(
    {
        "Importance": grid_search.best_estimator_.feature_importances_,
        "Feature": df_features_prc.columns
    }
)
feature_importance.sort_values(by="Importance", ascending=False)
print(display(feature_importance))

ValueError: arrays must all be same length

---
---